In [1]:
import pandas as pd
import numpy as np
import datetime
import json
import requests
import time
import sys

    
from objsize import get_deep_size
from pympler import asizeof


In [9]:
url = "http://gbfs.citibikenyc.com/gbfs/gbfs.json"

data_urls = requests.get(url).json()
station_status_url =data_urls['data']['en']['feeds'][2]['url']

station_status_url

'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'

# Inspect single station data

In [11]:
station_status = requests.get(station_status_url).json()

a_station = station_status['data']['stations'][0]
a_station['station_id']
a_station['last_reported']
a_station
a_station.get('station_id')

'72'

In [23]:
data_dict = {}

try:
    data_dict['72']
except KeyError:
    print('no prob')
    data_dict['72'] = 'bla'
    
# data_dict.get('72')
 

data_dict

no prob


{'72': 'bla'}

# Instantiate the data dictionary

In [25]:
data_dict = {}
for station in station_status['data']['stations']:
    station_id = station['station_id']
    station_time = station['last_reported']
    
    subsubdict = {'num_bikes_available': station['num_bikes_available'], 
           'num_ebikes_available': station['num_ebikes_available'], 
            }
    subdict = {station_time: subsubdict}
    data_dict[station_id] = subdict 

# Request station data iteratively

In [30]:
data_dict = {}

for i in range(2):
    station_status = requests.get(station_status_url).json()
    
    for station in station_status['data']['stations']:
         
        station_id = station['station_id']
        station_time = station['last_reported']
        
        try:
            data_dict[station_id]
        except KeyError:
            data_dict[station_id] = {}
            
            
        if station_time not in data_dict[station_id]:
            subsubdict = {'num_bikes_available': station['num_bikes_available'], 
                   'num_ebikes_available': station['num_ebikes_available'], 
                    }
            subdict = {station_time: subsubdict}
            olddict = data_dict[station_id]
            
            data_dict[station_id] = {**olddict, **subdict}
    time.sleep(1)
    
    if i%1 == 0: 
        print(f"iteration: {i}, data_dict size: {asizeof.asizeof(data_dict)}" )
     
        

iteration: 0, data_dict size: 642032
iteration: 1, data_dict size: 642032


In [31]:
data_dict

{'72': {86400: {'num_bikes_available': 0, 'num_ebikes_available': 0}},
 '79': {1597793888: {'num_bikes_available': 14, 'num_ebikes_available': 0}},
 '82': {1597793890: {'num_bikes_available': 12, 'num_ebikes_available': 0}},
 '83': {1597793451: {'num_bikes_available': 47, 'num_ebikes_available': 0}},
 '116': {1597793987: {'num_bikes_available': 13, 'num_ebikes_available': 0}},
 '119': {1597788875: {'num_bikes_available': 19, 'num_ebikes_available': 0}},
 '120': {1597793831: {'num_bikes_available': 1, 'num_ebikes_available': 0}},
 '127': {1597793818: {'num_bikes_available': 31, 'num_ebikes_available': 1}},
 '128': {1597793986: {'num_bikes_available': 18, 'num_ebikes_available': 0}},
 '143': {1597792318: {'num_bikes_available': 0, 'num_ebikes_available': 0}},
 '144': {1597793959: {'num_bikes_available': 46, 'num_ebikes_available': 0}},
 '146': {1597793691: {'num_bikes_available': 14, 'num_ebikes_available': 1}},
 '150': {1597793791: {'num_bikes_available': 14, 'num_ebikes_available': 0}}

# Inspect dictionary

In [16]:
for key, item in data_dict['247'].items():
    print(datetime.datetime.fromtimestamp(key) )

2020-08-12 20:39:54


In [17]:
dict_of_df = {k: pd.DataFrame(v) for k,v in data_dict.items()}
df = pd.concat(dict_of_df, axis=1).transpose()
df

num_bikes_available  num_ebikes_available
72   86400                         0                     0
79   1597278788                   18                     0
     1597279401                   17                     0
82   1597278799                   17                     0
83   1597278877                   40                     0
...                              ...                   ...
4109 86400                         0                     0
4112 86400                         0                     0
4113 86400                         0                     0
4114 86400                         0                     0
4115 86400                         0                     0

[1960 rows x 2 columns]

# Save dictionary

In [13]:
with open("station_data.json", "w") as outfile:  
    json.dump(data_dict, outfile)  

In [22]:
now = datetime.datetime.now()

In [20]:
tomorrow_noon = datetime.datetime(2020,8,13,12,0,0)

In [33]:
dt = tomorrow_noon - now
# dt.days
dt.seconds / 3600/24 /7

0.08987929894179894

In [31]:
i = 0

while True:
    i = i+1
    
    print(i)
    if i >=10:
        break

1
2
3
4
5
6
7
8
9
10
